In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

In [6]:
import json

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
url = 'https://spb.hh.ru'
url_add = '/vacancies/data-scientist'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}
response = requests.get(url+url_add, headers=headers)
soup = bs(response.text, 'html.parser')
pager_next = soup.find('a',attrs={'class':'bloko-button', 'data-qa':'pager-next'}).get('href')
jobs = []
job_list = soup.find_all('div',attrs={'class':'vacancy-serp-item'})

In [5]:
try:
    while pager_next:
        for job in job_list:
            job_data = {}
            job_link = job.find('a',attrs={'class':'bloko-link', 'data-qa':'vacancy-serp__vacancy-title'}).get('href')
            job_name = job.find('a',attrs={'class':'bloko-link'}).getText()
            job_salary = job.find('span',attrs={'data-qa': 'vacancy-serp__vacancy-compensation', 'class':'bloko-header-section-3 bloko-header-section-3_lite'})

            if job_salary:
                if job_salary.getText().startswith('от'):
                    job_salary_min = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                    job_salary_max = None
                    job_currency = job_salary.getText().split(' ')[2]
                    
                elif job_salary.getText().startswith('до'):
                    job_salary_min = None
                    job_salary_max = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                    job_currency = job_salary.getText().split(' ')[2]
                    
                else:
                    job_salary_min = int(job_salary.getText().split(' ')[0].replace('\u202f', ''))
                    job_salary_max = int(job_salary.getText().split(' ')[2].replace('\u202f', ''))
                    job_currency = job_salary.getText().split(' ')[3]
            
            else:
                job_salary_min = None
                job_salary_max = None
                job_currency = None
                
            job_data['link'] = job_link
            job_data['name'] = job_name
            job_data['salary_min'] = job_salary_min
            job_data['salary_max'] = job_salary_max
            job_data['currency'] = job_currency

            jobs.append(job_data)
                
        response = requests.get(url+pager_next, headers=headers)
        soup = bs(response.text, 'html.parser')
        job_list = soup.find_all('div',attrs={'class':'vacancy-serp-item'})
        pager_next = soup.find('a',attrs={'class':'bloko-button', 'data-qa':'pager-next'}).get('href')
        
except AttributeError:
    for job in job_list:
        job_data = {}
        job_link = job.find('a',attrs={'class':'bloko-link', 'data-qa':'vacancy-serp__vacancy-title'}).get('href')
        job_name = job.find('a',attrs={'class':'bloko-link'}).getText()
        job_salary = job.find('span',attrs={'data-qa': 'vacancy-serp__vacancy-compensation', 'class':'bloko-header-section-3 bloko-header-section-3_lite'})

        if job_salary:
            if job_salary.getText().startswith('от'):
                job_salary_min = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                job_salary_max = None
                job_currency = job_salary.getText().split(' ')[2]
                
            elif job_salary.getText().startswith('до'):
                job_salary_min = None
                job_salary_max = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                job_currency = job_salary.getText().split(' ')[2]
                
            else:
                job_salary_min = int(job_salary.getText().split(' ')[0].replace('\u202f', ''))
                job_salary_max = int(job_salary.getText().split(' ')[2].replace('\u202f', ''))
                job_currency = job_salary.getText().split(' ')[3]
                
                
        else:
            job_salary_min = None
            job_salary_max = None
            job_currency = None
            
        job_data['link'] = job_link
        job_data['name'] = job_name
        job_data['salary_min'] = job_salary_min
        job_data['salary_max'] = job_salary_max
        job_data['currency'] = job_currency

        jobs.append(job_data)
    
    pprint(jobs)

[{'currency': None,
  'link': 'https://spb.hh.ru/vacancy/46702571?from=vacancy_search_catalog&query=data%20scientist',
  'name': 'Data scientist | Специалист по машинному обучению (junior/middle)',
  'salary_max': None,
  'salary_min': None},
 {'currency': 'руб.',
  'link': 'https://spb.hh.ru/vacancy/46590676?from=vacancy_search_catalog&query=data%20scientist',
  'name': 'Специалист Computer Vision (Senior Data Scientist)',
  'salary_max': 300000,
  'salary_min': 180000},
 {'currency': 'руб.',
  'link': 'https://spb.hh.ru/vacancy/46732109?from=vacancy_search_catalog&query=data%20scientist',
  'name': 'Senior Data Scientist',
  'salary_max': 320000,
  'salary_min': 220000},
 {'currency': None,
  'link': 'https://spb.hh.ru/vacancy/46750731?from=vacancy_search_catalog&query=data%20scientist',
  'name': 'Аналитик данных / Data scientist',
  'salary_max': None,
  'salary_min': None},
 {'currency': 'руб.',
  'link': 'https://spb.hh.ru/vacancy/45260586?from=vacancy_search_catalog&query=data%2

In [9]:
jobs_df = pd.DataFrame(jobs)

In [11]:
jobs_df.to_csv('data_sci.csv', sep=',', index=False)